# Interval Newton Methods prototype

In [1]:
import numpy as np
from sympy import *
init_printing()
# import math
from math import *
from interval import interval
from interval import imath as im

## Sympy

### Sympyで関数定義

In [2]:
x,c = symbols("x c")
f = symbols('f', cls=Function)
f = x**3 - 40*x**2 +3000 
#f = x**2 - c 
#f = tan((float(3)/2)*pi*x) - sqrt(sin(x)**2 - (float(9)/4)**(-1)) / cos(x)
# init_printing(pretty_print = False)
print str(f)

f

x**3 - 40*x**2 + 3000


### fの微分

In [3]:
f.diff(x)

### symbolを文字列化して評価

In [4]:
func = lambda x: eval(str(f))
deriv_eval_str = str(f.diff(x))
deriv_func = lambda x: eval(deriv_eval_str)

print func
print deriv_func

#例
print func(5)
print deriv_func(10)

<function <lambda> at 0x105c3bf50>
<function <lambda> at 0x105ccb0c8>
2125
-500


## [PyInterval](http://pyinterval.readthedocs.io/en/latest/_modules/interval.html#interval.newton)

### cheet sheet

In [5]:
sita = interval[-2,3]
print sita
print im.cos(sita)

a = interval([0, 1], [2, 3], [10, 15])
print a
for p in a:
    print "p : " + str(p)
    for q in p:
        print "q : " + str(q)
    print '-----'*5
    

interval([-2.0, 3.0])
interval([-0.9899924966, 1.0])
interval([0.0, 1.0], [2.0, 3.0], [10.0, 15.0])
p : (0.0, 1.0)
q : 0.0
q : 1.0
-------------------------
p : (2.0, 3.0)
q : 2.0
q : 3.0
-------------------------
p : (10.0, 15.0)
q : 10.0
q : 15.0
-------------------------


## 区間ニュートン法
$X^{k+1} = X^{k} \cap N(X^{k}) (k = 0,1,...)$,
where
$N(X) = m(X) - f(m(X))/F^{'}(X) $


### 自分でnewton法を実装

In [6]:
def interval_newton_method(func, deriv_func, current, maxiter=100, trace=False,depth=0):
    def extract_pivot(i):
        yield i.midpoint
        for x in i:
            for num in x:
                yield interval(num)

    def refine_by_intersection(pivot, interval):
            N = pivot - func(pivot) / deriv_func(interval)
            return N & interval
            
    prefix = ("|" + " "*7)*depth
    if trace: print prefix +"-- start -- current : %s" % str(current)
    for epoch in xrange(maxiter):
        if trace: print prefix + "epoch : " + str(epoch)
        prev = current
        for i, p in enumerate(extract_pivot(current)):
            current = refine_by_intersection(p,current)
            if trace: print prefix + "pivot index : " + str(i) + ", current after intersection : " + str(current) 
            if current == prev: next
            else: break
        else:#これ以上繰り返す必要なし
            return current 
        if len(current) > 1: #再帰
            res = [interval_newton_method(func,deriv_func, interval[val], maxiter, trace, depth+1) for val in current]
            res = reduce(lambda x,y: x|y, res)
            return res
        elif current == interval(): # これ以上繰り返す必要なし
            return current
    return current

### 例1

In [7]:
func = lambda x: eval(str(f))
deriv_eval_str = str(f.diff(x))
deriv_func = lambda x: eval(deriv_eval_str)
f

#### 事前にsympyで答えを把握

In [8]:
var("x")      # a～zまで変数として扱う
eq = Eq(eval(str(f)), 0)   # 方程式の定義
ans = solve(eq)         # 方程式の解を計算
print ans
symbols("x")

calc_ans = [eval(str(a)) for a in ans]
calc_ans


[10, 15 + 5*sqrt(21), -5*sqrt(21) + 15]


#### pyintervalで区間ニュートン法

In [9]:
X_0 = interval[-14,155]

def tracer_cb(x,y):
    pass

pyinterval_ans = X_0.newton(func, deriv_func, maxiter = 100,tracer_cb=tracer_cb)
pyinterval_ans

#### 自作の区間ニュートン法

In [10]:
 interval_newton_method(func,deriv_func,X_0,maxiter=100,trace=False)

### p113. Example 8.3 $f(x)=x^2 - c $, for $c \in [2,3]$

In [11]:
func = lambda x: x**2 - interval[2,3]
deriv_func = lambda x: 2*x
X_0 = interval[-10,20]

print "--"*5+"---結果の比較"+"--"*15
print '-- 自作newton'
print interval_newton_method(func,deriv_func,X_0,maxiter=100,trace=False)
# print interval_newton_method(func,deriv_func,X_0,maxiter=100,trace=True)
print
print '-- pyinterval newton'
print X_0.newton(func,deriv_func)


-------------結果の比較------------------------------
-- 自作newton
interval([-1.73205080757, -1.41421356237], [1.41421356237, 1.73205080757])

-- pyinterval newton
interval([-1.73205080757, -1.41421356237], [1.41421356237, 1.73205080757])


### $f = cos(x)$

In [12]:
func = lambda x: im.cos(x)
deriv_func = lambda x: -im.sin(x)

X_0 = interval[-5,2]
print X_0.newton(func,deriv_func)
print 
print interval_newton_method(func,deriv_func,X_0)


interval([-4.71238898038, -4.71238898038], [-1.57079632679, -1.57079632679], [1.57079632679, 1.57079632679])

interval([-4.71238898038, -4.71238898038], [-1.57079632679, -1.57079632679], [1.57079632679, 1.57079632679])
